<a href="https://colab.research.google.com/github/KauaHenSilva/python_predict_cat_or_dog/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Recuperação por download

In [22]:
# @markdown #### Caso escolheu o tipo de recuperação drive/ambos e não tenha os arquivos, faça um atalho desse drive:
link_do_drive = "https://drive.google.com/drive/folders/1qUk_XflSRRvw6PKzd293pNk8Bggl81PF?usp=drive_link" #@param {type:"string"}
path_atalho_drive = "/content/drive/MyDrive/Projetos Do GitHub/python_predict_cat_or_dog" #@param {type:"string"}
path_download = "python_predict_cat_or_dog" #@param {type:"string"}

In [23]:
import os
import zipfile

if os.path.exists('python_download_file_drive'):
  !cd python_download_file_drive && git pull
else:
  !git clone https://github.com/KauaHenSilva/python_download_file_drive
%pip install --upgrade --quiet -r python_download_file_drive/requirements.txt


Already up to date.


In [24]:
try:
    from google.colab import drive
    from google.colab import auth
    from googleapiclient.discovery import build
    auth.authenticate_user()

    !python "python_download_file_drive/main.py" {link_do_drive} {path_download} # ALERTA: Caso o drive bote limite de download comente essa linha

    drive.mount('/content/drive')
    if not os.path.exists(path_atalho_drive):
      print("[INFO] Atalho no drive não encontrado.")

except:

    if os.path.exists(f"{path_download}.zip"):
      print(f"Descompactando {path_download}.zip")
      !rm -rf {path_download}
      with zipfile.ZipFile(f"{path_download}.zip", 'r') as zip_ref:
        zip_ref.extractall(f"{path_download}_temp")
        !mv {path_download}_temp/* {path_download}
        !rm -rf {path_download}_temp
        !rm -rf {path_download}.zip

    if not os.path.exists(path_download):
      raise Exception(f"Você não está nos altenticado nos servidores do google colab. Para prosseguir, faça o download dos arquivos que estão presente no drive e tente novamente. O link do drive {link_do_drive}. A pasta deve está dentro de /content/")


In [25]:
def get_referencia_valida(path, file_init):
    path_download = os.path.join(f"/content/{file_init}", path)
    path_drive = os.path.join(path_atalho_drive, path)

    bool_download = os.path.exists(path_download)
    bool_drive = os.path.exists(path_drive)

    if not bool_download and not bool_drive:
        raise Exception(f"Arquivo não encontrado em nenhum dos caminhos (Download ou Drive). Caminhos: {path_download}, {path_drive}")

    if not bool_drive and bool_download:
        print("[INFO] Arquivo encontrado no download, mas não no drive.")
        return path_download

    if bool_drive and not bool_download:
        print("[INFO] Arquivo encontrado no drive, mas não no download.")
        return path_drive

    return path_download


# Cat and Dog

In [26]:
import zipfile
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import os
import tensorflow as tf
import sklearn as sk

In [27]:
cv2.__version__, tf.__version__, sk.__version__

('4.10.0', '2.16.1', '1.6.0')

In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [29]:
path_database_zip = get_referencia_valida("dataset.zip", path_download)

[INFO] Arquivo encontrado no download, mas não no drive.


In [30]:
with zipfile.ZipFile(path_database_zip, 'r') as zip_ref:
    zip_ref.extractall('database')

In [31]:
path_dataset ='/content/database/dataset'

In [32]:
gerador_treinamento = ImageDataGenerator(rescale=1./255, rotation_range=7, horizontal_flip = True, shear_range=0.2, height_shift_range = 0.07, zoom_range=0.2)
gerador_teste = ImageDataGenerator(rescale=1./255)

In [33]:
base_treinamento = gerador_treinamento.flow_from_directory(f"{path_dataset}/training_set", target_size=(128, 128), batch_size=32, class_mode='binary')
base_teste = gerador_teste.flow_from_directory(f"{path_dataset}/test_set", target_size=(128, 128), batch_size=32, class_mode='binary')

Found 4000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [34]:
net = Sequential()
net.add(InputLayer(shape=(128, 128, 3)))
net.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
net.add(BatchNormalization())
net.add(MaxPooling2D((2, 2)))
net.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
net.add(BatchNormalization())
net.add(MaxPooling2D((2, 2)))
net.add(Flatten())
net.add(Dense(256, activation='relu'))
net.add(Dropout(0.2))
net.add(Dense(256, activation='relu'))
net.add(Dropout(0.2))
net.add(Dense(1, activation='sigmoid'))

In [35]:
net.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)                    │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 126, 126, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 61, 61, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 61, 61, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 30, 30, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 28800)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 256)                 │       7,373,056 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 256)                 │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,449,505 (28.42 MB)

 Trainable params: 7,449,377 (28.42 MB)

 Non-trainable params: 128 (512.00 B)

In [36]:
net.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [37]:
net.fit(base_treinamento, epochs=10, validation_data=base_teste);

Epoch 1/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 19s 110ms/step - accuracy: 0.5535 - loss: 2.2056 - val_accuracy: 0.5000 - val_loss: 2.4765
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 14s 107ms/step - accuracy: 0.5884 - loss: 0.8935 - val_accuracy: 0.5050 - val_loss: 2.5719
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 14s 105ms/step - accuracy: 0.6308 - loss: 0.7029 - val_accuracy: 0.5750 - val_loss: 0.8154
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 14s 105ms/step - accuracy: 0.6321 - loss: 0.6618 - val_accuracy: 0.6300 - val_loss: 0.7655
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 14s 105ms/step - accuracy: 0.6455 - loss: 0.6465 - val_accuracy: 0.6210 - val_loss: 0.7670
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 13s 102ms/step - accuracy: 0.6473 - loss: 0.6238 - val_accuracy: 0.6910 - val_loss: 0.6115
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 13s 101ms/step - accuracy: 0.6807 - loss: 0.6061 - val_accuracy: 0.6600 - val_loss: 0.6539
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 13s 101ms/step - accuracy: 0.6877 - loss: 0

In [38]:
net.evaluate(base_teste)

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.6993 - loss: 0.5718


[0.5721911191940308, 0.7059999704360962]